## 한국음식 분류기

In [74]:
## 사용할 모델 라이브러리 import
import sys, os
#from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Activation
from keras.layers import Dropout
from keras.layers import Flatten
#from keras.layers import Dense
#from keras.utils import np_utils
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical  
import pickle

In [75]:
## 초기 설정
root_dir = "./kfood/"
categories = ["Chicken", "Dolsotbab", "Jeyugbokkeum", "Kimchi", 
                "Samgyeobsal", "SoybeanPasteStew"]
nb_classes = len(categories)
image_size = 128

In [76]:
# 데이터 로딩  (1)
def load_dataset():
    with open(root_dir + "kfood.pkl", "rb") as f:
        x_train, x_test, y_train, y_test = pickle.load(f)
   # x_train, x_test, y_train, y_test = np.load("./kfood/kfood.npy", allow_pickle=True)
    x_train = x_train.astype("float") / 256
    x_test = x_test.astype("float") / 256
    y_train = to_categorical(y_train, nb_classes)
    y_test = to_categorical(y_test, nb_classes)
    return  x_train, x_test, y_train, y_test

In [77]:
# 모델 구성  (2)
# padding='same' : 출력 이미지 사이즈가 입력 이미지 사이즈와 동일
# ‘valid’인 경우 출력 이미지 크기가 입력 이미지 크기보다 작아짐
# 참조 : https://tykimos.github.io/2017/01/27/CNN_Layer_Talk/
def build_model(in_shape):
    model = Sequential()
    model.add(Convolution2D(32, 3, 3, padding='same', 
                input_shape=in_shape))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(1,1)))
    model.add(Dropout(0.25))
    
    model.add(Convolution2D(64, 3, 3, padding='same'))
    model.add(Activation('relu'))
    
    model.add(Convolution2D(64, 3, 3))
    model.add(MaxPooling2D(pool_size=(1,1)))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    
    model.add(Dense(nb_classes))
    model.add(Activation('softmax'))
    model.compile(loss='binary_crossentropy', 
                    optimizer='rmsprop', 
                    metrics=['accuracy'])

    return model

In [78]:
# 모델 학습을 수행하고 저장된 모델을 파일로 저장  (3)
def model_train(x, y):
    model = build_model(x.shape[1:])
    model.fit(x, y, batch_size=32, epochs=30)

    return model

In [79]:
# 모델 평가하기  (4)
def model_eval(model, x, y):
    score = model.evaluate(x, y)
    print('loss=', score[0])
    print('accuracy=', score[1])

In [80]:
# 모델 학습 및 평가
x_train, x_test, y_train, y_test = load_dataset()

In [81]:
x_train.shape

(133, 128, 128, 3)

In [82]:
y_train[0]

array([1., 0., 0., 0., 0., 0.])

In [83]:
y_train.shape

(133, 6)

In [84]:
model = model_train(x_train, y_train)
model_eval(model, x_test, y_test)

Epoch 1/30
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.1550 - loss: 0.6570
Epoch 2/30
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.1833 - loss: 0.4782
Epoch 3/30
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.2431 - loss: 0.4513
Epoch 4/30
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.2852 - loss: 0.4627
Epoch 5/30
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.2599 - loss: 0.4456
Epoch 6/30
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.3066 - loss: 0.4374
Epoch 7/30
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.3016 - loss: 0.4347
Epoch 8/30
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.3516 - loss: 0.4098
Epoch 9/30
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.4167 - loss: 0.4033
Epoch 10/30
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.4459 - loss: 0.3729
Epoch 11/30
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.5451 - loss: 0.3588
Epoch 12/30
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.4630 - loss: 0.3443
E

In [85]:
# 모델 저장
model.save("./kfood/kfood_model.h5")